In [1]:
import copy
import json
import datetime
import os
import types
import warnings

import matplotlib.patches as patches
import matplotlib.pyplot as plt
import torch
import cv2
import numpy as np
from mmengine.config import Config
from mmengine.runner import Runner
from mmengine.hooks import EarlyStoppingHook, Hook
from mmdet.apis import inference_detector, init_detector
from mmdet.registry import MODELS
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
import wandb
from pycocotools.coco import COCO


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

class InitHook(Hook):
    def before_run(self, runner):
        """
        모델의 평가 설정을 초기화하는 Hook.
        cfg를 SimpleNamespace로 변환하지 않고 mmengine의 Config 객체를 유지합니다.
        """
        # eval_module에 따라 test_cfg 설정
        eval_module = runner.cfg.model.eval_module
        if eval_module == 'detr':
            # 필요에 따라 추가 설정 가능
            pass
        elif eval_module == 'one-stage':
            pass
        elif eval_module == 'two-stage':
            pass
        else:
            raise ValueError(f"Unknown eval_module: {eval_module}")
        
        # 필수 필드가 누락되지 않도록 확인 및 추가
        if not hasattr(runner.cfg.model.test_cfg, 'score_thr'):
            runner.cfg.model.test_cfg.score_thr = 0.05  # 기본값 설정
            print("InitHook: Added 'score_thr' to test_cfg with default value 0.05.")
        
        print("InitHook: runner.cfg.model.test_cfg has been set.")
        print(f"runner.cfg.model.test_cfg: {runner.cfg.model.test_cfg}")

In [3]:
class TopMisclassifiedImagesHook(Hook):
    def __init__(self, dataset, ann_file, work_dir, top_k=20, score_thr=0.3):
        super().__init__()
        self.dataset = dataset
        self.ann_file = ann_file
        self.work_dir = work_dir
        self.top_k = top_k
        self.score_thr = score_thr

    def after_val_epoch(self, runner, **kwargs):
        print("TopMisclassifiedImagesHook: after_val_epoch called")
        print(f"runner.cfg.model.test_cfg: {runner.cfg.model.test_cfg}")
        misclassified = []

        # 데이터셋 순회
        for idx in range(len(self.dataset)):
            try:
                data = self.dataset[idx]
            except Exception as e:
                print(f"Error accessing dataset index {idx}: {e}")
                continue

            # 첫 번째 샘플의 데이터 구조 출력 (디버깅 용도)
            if idx == 0:
                print(f"Data structure example: {data}")
                print(f"Metainfo: {self.dataset.metainfo}")

            # data_samples 추출
            data_samples = data.get('data_samples', None)
            if data_samples is None:
                print(f"Warning: data_samples is missing for index {idx}")
                continue

            # img_path는 data_samples의 속성으로 접근
            img_path = getattr(data_samples, 'img_path', '')
            if not img_path:
                print(f"Warning: img_path is missing in data_samples for index {idx}")
                continue

            img = cv2.imread(img_path)
            if img is None:
                print(f"Warning: Image {img_path} could not be read.")
                continue

            # Ground Truth 추출
            gt_instances = getattr(data_samples, 'gt_instances', None)
            if gt_instances is None:
                print(f"Warning: gt_instances is missing in data_samples for index {idx}")
                continue

            gt_labels = getattr(gt_instances, 'labels', [])
            gt_bboxes = getattr(gt_instances, 'bboxes', [])

            # 텐서를 리스트나 넘파이 배열로 변환
            if isinstance(gt_labels, torch.Tensor):
                gt_labels = gt_labels.cpu().numpy().tolist()
            if isinstance(gt_bboxes, torch.Tensor):
                gt_bboxes = gt_bboxes.cpu().numpy().tolist()

            if not gt_labels or not gt_bboxes:
                print(f"Warning: No ground truth labels or bboxes for image {img_path}.")
                continue

            # 모델 추론 - test_pipeline을 전달하지 않음
            try:
                results = inference_detector(runner.model, img)
            except Exception as e:
                print(f"Error during inference for image {img_path}: {e}")
                continue

            # 결과 처리 (bbox 결과 가정)
            if isinstance(results, tuple):
                bbox_result = results[0]
            else:
                bbox_result = results

            # 예측된 클래스 라벨 수집
            pred_labels = []
            for class_id, bboxes in enumerate(bbox_result, start=1):
                for bbox in bboxes:
                    pred_labels.append(class_id)

            # Ground Truth와 예측된 라벨 비교
            gt_set = set(gt_labels)
            pred_set = set(pred_labels)
            misclass = not gt_set.issubset(pred_set)

            if misclass:
                # 오분류 정도를 나타내는 오류 점수 계산 (대칭 차집합의 크기)
                error_score = len(gt_set.symmetric_difference(pred_set))
                misclassified.append((error_score, img_path, img, results, gt_bboxes, gt_labels))

        if not misclassified:
            print("TopMisclassifiedImagesHook: No misclassified images found.")
            return

        # 오류 점수를 기준으로 내림차순 정렬
        misclassified = sorted(misclassified, key=lambda x: x[0], reverse=True)
        top_misclassified = misclassified[:self.top_k]

        # 상위 오분류된 이미지를 WandB에 로그
        for error_score, img_path, img, results, gt_bboxes, gt_labels in top_misclassified:
            # 이미지에 예측 결과와 Ground Truth를 그려줍니다.
            fig, ax = plt.subplots(1, figsize=(12, 12))
            ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

            # Ground Truth 바운딩 박스 그리기
            for bbox, label in zip(gt_bboxes, gt_labels):
                x1, y1, x2, y2 = bbox
                width, height = x2 - x1, y2 - y1
                rect = patches.Rectangle((x1, y1), width, height, linewidth=2, edgecolor='g', facecolor='none')
                ax.add_patch(rect)
                ax.text(x1, y1 - 5, f"GT: {self.dataset.metainfo['classes'][label - 1]}", color='green', fontsize=12, backgroundcolor='black')

            # 예측된 바운딩 박스 그리기
            for class_id, bboxes in enumerate(results, start=1):
                for bbox in bboxes:
                    score = bbox[-1]
                    if score < self.score_thr:
                        continue
                    x1, y1, x2, y2 = bbox[:4]
                    width, height = x2 - x1, y2 - y1
                    rect = patches.Rectangle((x1, y1), width, height, linewidth=2, edgecolor='r', facecolor='none')
                    ax.add_patch(rect)
                    ax.text(x1, y1 - 5, f"{self.dataset.metainfo['classes'][class_id - 1]}: {score:.2f}", color='yellow', fontsize=12, backgroundcolor='black')

            ax.axis('off')

            # 시각화된 이미지를 파일로 저장
            vis_path = os.path.join(self.work_dir, f"misclassified_{os.path.basename(img_path)}.png")
            fig.savefig(vis_path, bbox_inches='tight', pad_inches=0)
            plt.close(fig)

            # WandB에 시각화된 이미지 로그
            try:
                runner.logger.experiment.log({
                    "Top Misclassified Images": wandb.Image(vis_path, caption=f"Path: {img_path} | Error Score: {error_score}")
                })
                print(f"Logged misclassified image: {vis_path} to WandB")
            except Exception as e:
                print(f"Error logging to WandB: {e}")

        print(f"WandB에 상위 {self.top_k}개의 오분류 이미지를 로그했습니다.")

In [4]:
# 설정 파일 로드
config_path = './projects/CO-DETR/configs/codino/co_dino_5scale_swin_l_16xb1_1x_coco.py'  # DINO 설정 파일 경로
cfg = Config.fromfile(config_path)

# 작업 디렉토리 설정
work_dir = './work_dirs/co_dino_custom'  # 로그와 모델을 저장할 디렉토리 경로
cfg.work_dir = work_dir

In [5]:
# 클래스 설정
classes = (
    "General trash", "Paper", "Paper pack", "Metal", "Glass",
    "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"
)
num_classes = len(classes)  # 10

In [6]:
# 학습 및 검증 데이터셋 분할 (멀티라벨 스트라티파이드 분할)
original_ann_file = '/data/ephemeral/home/dataset/train.json'  # 전체 데이터 어노테이션 파일 경로

with open(original_ann_file, 'r') as f:
    annotations = json.load(f)

# 이미지 ID 리스트 가져오기
image_ids = [image['id'] for image in annotations['images']]

# 이미지 ID와 해당 이미지에 포함된 클래스 목록 매핑
image_id_to_classes = {image_id: [] for image_id in image_ids}
for ann in annotations['annotations']:
    image_id = ann['image_id']
    category_id = ann['category_id']
    if category_id not in image_id_to_classes[image_id]:
        image_id_to_classes[image_id].append(category_id)

# 멀티라벨 인코딩을 위한 이진 매트릭스 생성
label_matrix = np.zeros((len(image_ids), num_classes))
image_id_to_index = {image_id: idx for idx, image_id in enumerate(image_ids)}
for image_id, class_ids in image_id_to_classes.items():
    idx = image_id_to_index[image_id]
    for class_id in class_ids:
        label_matrix[idx, class_id - 1] = 1  # category_id는 1부터 시작

# 멀티라벨 스트라티파이드 셔플 스플릿 사용
msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.9, random_state=42)  # 80:20 분할
train_indices, val_indices = next(msss.split(image_ids, label_matrix))

# 인덱스를 이미지 ID로 변환
train_ids = [image_ids[idx] for idx in train_indices]
val_ids = [image_ids[idx] for idx in val_indices]

# 학습 및 검증 데이터 어노테이션 생성
train_annotations = {
    'images': [img for img in annotations['images'] if img['id'] in train_ids],
    'annotations': [ann for ann in annotations['annotations'] if ann['image_id'] in train_ids],
    'categories': annotations['categories']
}
val_annotations = {
    'images': [img for img in annotations['images'] if img['id'] in val_ids],
    'annotations': [ann for ann in annotations['annotations'] if ann['image_id'] in val_ids],
    'categories': annotations['categories']
}

# 분할된 어노테이션을 파일로 저장
train_ann_file = '/data/ephemeral/home/dataset/train_split.json'
val_ann_file = '/data/ephemeral/home/dataset/val_split.json'

with open(train_ann_file, 'w') as f:
    json.dump(train_annotations, f)
with open(val_ann_file, 'w') as f:
    json.dump(val_annotations, f)


In [7]:
# 어노테이션 파일 수정: category_id를 1부터 시작하도록 조정
def increment_category_id(annotation_file):
    with open(annotation_file, 'r') as f:
        data = json.load(f)

    # 'categories' 섹션의 'id'를 1부터 시작하도록 수정
    id_mapping = {}
    for category in data['categories']:
        old_id = category['id']
        new_id = old_id + 1
        id_mapping[old_id] = new_id
        category['id'] = new_id

    # 'annotations' 섹션의 'category_id'를 매핑에 따라 수정
    for ann in data['annotations']:
        old_cat_id = ann['category_id']
        if old_cat_id in id_mapping:
            ann['category_id'] = id_mapping[old_cat_id]
        else:
            print(f"Warning: annotation id {ann['id']} has invalid category_id {old_cat_id}")

    # 수정된 데이터를 원래 파일에 저장
    with open(annotation_file, 'w') as f:
        json.dump(data, f, indent=4)

# 어노테이션 파일에 category_id를 1부터 시작하도록 수정
increment_category_id(train_ann_file)
increment_category_id(val_ann_file)

In [8]:
# EarlyStoppingHook 추가
early_stopping_hook = dict(
    type='EarlyStoppingHook',
    monitor='bbox_mAP',
    rule='greater',
    min_delta=0.001,
    patience=5,
    check_finite=True,
    stopping_threshold=None
)

# cfg.default_hooks에 EarlyStoppingHook 추가
cfg.default_hooks.update(
    early_stopping=early_stopping_hook
)

# WandB 초기화 (Runner 전에)
run_name = f'co_dino_5scale_swin_l_lsj_16xb1_1x_coco_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}'
wandb.init(
    project='Object_detection',
    name=run_name,
    config=cfg.to_dict(),
    allow_val_change=True,
    reinit=True
)
print(f"WandB Run Initialized: {run_name}")

# WandbVisBackend 설정
wandb_vis_backend = dict(
    type='WandbVisBackend',
    save_dir=cfg.work_dir,  # 저장할 디렉토리
    init_kwargs=dict(
        project='Object_detection',  # WandB 프로젝트 이름
        name=run_name,                # 고유한 실행 이름
        allow_val_change=True         # 설정 값 변경 허용
    ),
    define_metric_cfg=None,
    commit=True,
    log_code_name=None,
    watch_kwargs=None
)

# Visualizer 설정을 Runner의 visualizer 필드로 추가
cfg.visualizer = dict(
    type='Visualizer',
    vis_backends=[
        wandb_vis_backend  # WandbVisBackend 추가
    ],
    name='visualizer'  # Visualizer 이름 (옵션)
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


WandB Run Initialized: co_dino_5scale_swin_l_lsj_16xb1_1x_coco_20241013_195240


In [9]:
# 데이터셋 루트 경로 수정 및 데이터 파이프라인 수정
train_data_root = '/data/ephemeral/home/dataset/'  # 학습 데이터 경로
cfg.train_dataloader.dataset.ann_file = train_ann_file  # 수정된 부분
cfg.train_dataloader.dataset.data_prefix = dict(img=train_data_root)  # 수정된 부분

cfg.val_dataloader.dataset.ann_file = val_ann_file  # 수정된 부분
cfg.val_dataloader.dataset.data_prefix = dict(img=train_data_root)  # 수정된 부분

# 클래스 설정을 데이터셋의 metainfo에 추가
cfg.train_dataloader.dataset.metainfo = dict(classes=classes)  # 수정된 부분
cfg.val_dataloader.dataset.metainfo = dict(classes=classes)    # 수정된 부분

# 데이터 파이프라인 수정: 마스크 관련 부분 제거
for pipeline in cfg.train_dataloader.dataset.pipeline:
    if pipeline['type'] == 'LoadAnnotations':
        pipeline['with_mask'] = False

for pipeline in cfg.val_dataloader.dataset.pipeline:
    if pipeline['type'] == 'LoadAnnotations':
        pipeline['with_mask'] = False

# 클래스 수 수정
# bbox_head가 리스트인 경우 모든 헤드에 대해 num_classes 설정
if isinstance(cfg.model.bbox_head, list):
    for head in cfg.model.bbox_head:
        head['num_classes'] = num_classes
else:
    cfg.model.bbox_head['num_classes'] = num_classes

# roi_head 내의 bbox_head도 설정
if isinstance(cfg.model.roi_head, list):
    for roi_head in cfg.model.roi_head:
        if hasattr(roi_head, 'bbox_head'):
            roi_head.bbox_head.num_classes = num_classes
else:
    if hasattr(cfg.model.roi_head, 'bbox_head'):
        cfg.model.roi_head.bbox_head.num_classes = num_classes

# query_head도 num_classes 설정
if isinstance(cfg.model.query_head, list):
    for q_head in cfg.model.query_head:
        q_head.num_classes = num_classes
else:
    cfg.model.query_head.num_classes = num_classes

In [10]:

# 추가: 모델의 모든 관련 헤드에 num_classes가 올바르게 설정되었는지 확인
def verify_num_classes(cfg, num_classes):
    # bbox_head
    if isinstance(cfg.model.bbox_head, list):
        for head in cfg.model.bbox_head:
            assert head.num_classes == num_classes, "bbox_head num_classes 설정 오류"
    else:
        assert cfg.model.bbox_head.num_classes == num_classes, "bbox_head num_classes 설정 오류"

    # roi_head.bbox_head
    if isinstance(cfg.model.roi_head, list):
        for roi_head in cfg.model.roi_head:
            if hasattr(roi_head, 'bbox_head'):
                assert roi_head.bbox_head.num_classes == num_classes, "roi_head.bbox_head num_classes 설정 오류"
    else:
        if hasattr(cfg.model.roi_head, 'bbox_head'):
            assert cfg.model.roi_head.bbox_head.num_classes == num_classes, "roi_head.bbox_head num_classes 설정 오류"

    # query_head
    if isinstance(cfg.model.query_head, list):
        for q_head in cfg.model.query_head:
            assert q_head.num_classes == num_classes, "query_head num_classes 설정 오류"
    else:
        assert cfg.model.query_head.num_classes == num_classes, "query_head num_classes 설정 오류"

    print("모든 관련 헤드에 대해 num_classes가 올바르게 설정되었습니다.")

verify_num_classes(cfg, num_classes)


모든 관련 헤드에 대해 num_classes가 올바르게 설정되었습니다.


In [11]:
# 자동 혼합 정밀도 학습 사용 설정
use_amp = False  # 자동 혼합 정밀도(AMP) 학습 사용 여부
if use_amp:
    cfg.optim_wrapper.type = 'AmpOptimWrapper'
    cfg.optim_wrapper.loss_scale = 'dynamic'

# 학습률 자동 스케일링 설정
auto_scale_lr = False  # 학습률 자동 스케일링 사용 여부
if auto_scale_lr:
    if 'auto_scale_lr' in cfg and 'enable' in cfg.auto_scale_lr and 'base_batch_size' in cfg.auto_scale_lr:
        cfg.auto_scale_lr.enable = True
    else:
        raise RuntimeError('설정 파일에 "auto_scale_lr" 또는 필요한 키가 없습니다.')

In [12]:
# 데이터 로더 설정 최적화
cfg.train_dataloader.batch_size = 2  # 배치 사이즈를 2로 설정
cfg.val_dataloader.batch_size = 2

cfg.train_dataloader.num_workers = 2  # 워커 수 줄이기
cfg.val_dataloader.num_workers = 2

# Prefetch factor와 persistent_workers 설정
cfg.train_dataloader.prefetch_factor = 2
cfg.train_dataloader.persistent_workers = False
cfg.val_dataloader.prefetch_factor = 2
cfg.val_dataloader.persistent_workers = False

In [13]:

# val_evaluator 설정을 사용자 정의 검증 어노테이션 파일로 수정
cfg.val_evaluator = dict(
    type='CocoMetric',
    ann_file=val_ann_file,   # 사용자 정의 검증 어노테이션 파일 경로
    metric=['bbox']
)

# 또는, cfg.evaluation 섹션에 ann_file을 추가
if 'evaluation' in cfg:
    cfg.evaluation['ann_file'] = val_ann_file
else:
    cfg.evaluation = dict(
        ann_file=val_ann_file,
        interval=1,
        metric='bbox',
        save_best='auto'
    )

print("Evaluation 설정:")
print(cfg.val_evaluator)
print(cfg.evaluation)


Evaluation 설정:
{'type': 'CocoMetric', 'ann_file': '/data/ephemeral/home/dataset/val_split.json', 'metric': ['bbox']}
{'ann_file': '/data/ephemeral/home/dataset/val_split.json', 'interval': 1, 'metric': 'bbox', 'save_best': 'auto'}


In [14]:

# 러너 생성
runner = Runner.from_cfg(cfg)

# InitHook 등록 (높은 우선순위)
runner.register_hook(
    InitHook(),
    priority='VERY_HIGH'  # 매우 높은 우선순위로 설정
)

# TopMisclassifiedImagesHook 등록 (일반 우선순위)
runner.register_hook(
    TopMisclassifiedImagesHook(
        dataset=runner.val_dataloader.dataset,
        ann_file=val_ann_file,
        work_dir=work_dir,
        top_k=20,
        score_thr=0.3
    ),
    priority='NORMAL'
)


10/13 19:52:47 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 2057306621
    GPU 0: Tesla V100-SXM2-32GB
    CUDA_HOME: None
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.12.1+cu116
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.6
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-g

/data/ephemeral/home/baseline/mmdetection/projects/CO-DETR/codetr/transformer.py:1325: UserWarning: If you want to reduce GPU memory usage,                               please install fairscale by executing the                               following command: pip install fairscale.
  warnings.warn('If you want to reduce GPU memory usage, \
/data/ephemeral/home/baseline/mmdetection/mmdet/models/dense_heads/anchor_head.py:108: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


10/13 19:52:57 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
10/13 19:52:57 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
(LOWEST      ) EarlyStoppingHook                  
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_i

In [15]:
# 모든 레이어를 동결
for name, param in runner.model.named_parameters():
    param.requires_grad = False

# 필요한 레이어 Unfreeze
# bbox_head Unfreeze
for param in runner.model.bbox_head.parameters():
    param.requires_grad = True

# backbone의 마지막 두 레이어 Unfreeze
for name, param in runner.model.backbone.named_parameters():
    if 'stage3' in name or 'stage4' in name:
        param.requires_grad = True

# query_head Unfreeze
for name, param in runner.model.query_head.named_parameters():
    param.requires_grad = True

# roi_head Unfreeze
for name, param in runner.model.roi_head.named_parameters():
    param.requires_grad = True

# neck Unfreeze
for name, param in runner.model.neck.named_parameters():
    param.requires_grad = True

# rpn_head Unfreeze
for name, param in runner.model.rpn_head.named_parameters():
    param.requires_grad = True

# Optimizer 설정 수정
cfg.optim_wrapper.paramwise_cfg = dict(
    custom_keys={
        'backbone.stage3': dict(lr_mult=0.1),
        'backbone.stage4': dict(lr_mult=0.1),
        'neck': dict(lr_mult=1.0),
        'query_head': dict(lr_mult=1.0),
        'bbox_head': dict(lr_mult=1.0),
        'roi_head': dict(lr_mult=1.0),
        'rpn_head': dict(lr_mult=1.0)
    }
)

# 학습 시작 전에 검증 어노테이션 파일 존재 여부 확인
if not os.path.exists(val_ann_file):
    raise FileNotFoundError(f'Validation annotation file not found: {val_ann_file}')


In [16]:

# 학습 시작
runner.train()

# 학습 완료 후 단일 이미지 테스트 (선택 사항)
single_img_path = '/data/ephemeral/home/dataset/train/0003.jpg'
img = cv2.imread(single_img_path)
if img is not None:
    try:
        result = inference_detector(runner.model, img)
        print(f"Inference result for {single_img_path}: {result}")
    except Exception as e:
        print(f"Error during inference for {single_img_path}: {e}")
else:
    print(f"Image {single_img_path} could not be read.")

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
10/13 19:53:00 - mmengine - WARNING - backbone.patch_embed.projection.weight is skipped since its requires_grad=False
10/13 19:53:00 - mmengine - WARNING - backbone.patch_embed.projection.bias is skipped since its requires_grad=False
10/13 19:53:00 - mmengine - WARNING - backbone.patch_embed.norm.weight is skipped since its requires_grad=False
10/13 19:53:00 - mmengine - WARNING - backbone.patch_embed.norm.bias is skipped since its requires_grad=False
10/13 19:53:00 - mmengine - WARNING - backbone.stages.0.blocks.0.norm1.weight is skipped since its requires_grad=False
10/13 19:53:00 - mmengine - WARNING - backbone.stages.0.blocks.0.norm1.bias is skipped since its requires_grad=False
10/13 19:53:00 - mmengine - WARNING - backbone.stages.0.blocks.0.attn.w_msa.relative_position_bias_table is skipped since its requires_grad=False
10/13 19:53:00 - mmengine - WARNING - backbone.stages.0.blocks.0.attn.w_msa.qkv

AttributeError: 'list' object has no attribute 'score_thr'